In [45]:
!pip install tree-sitter tree-sitter-cpp


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import os

def load_code_from_files(directory_path):
    code_array = []

    # Iterate through all files in the directory
    for root, _, files in os.walk(directory_path):
        print(root, sorted(files))
        for file in sorted(files):
            # Only process files with a .cpp extension
            if file.endswith('.cpp') or file.endswith('.cc'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    code_content = f.read()
                    code_array.append(code_content)

    return code_array

def load_labels_from_files(directory_path):
    array = []

    # Iterate through all files in the directory
    for root, _, files in os.walk(directory_path):
        print(root, sorted(files))
        for file in sorted(files):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                array.append(content)

    return array

def get_names_of_files(labels_path):
    array = []

    # Iterate through all files in the directory
    for _, _, files in os.walk(labels_path):
        for file in sorted(files):
          array.append(file[:-4])
    return array

In [47]:
code_directory_path = '../CPP_Files'
labels_directory_path = '../Labels'
###REPLACE WITH CORRECT FILE PATHS

In [48]:
file_names = get_names_of_files(labels_directory_path)
print(file_names)

['addrdb', 'addresstype', 'addrman', 'auto_updater', 'command_line_args', 'electron_content_client', 'function_data', 'gpt_functions', 'gpt_se', 'gpt_sef', 'node_main', 'uv_stdio_fix']


In [49]:
# Step 4: Load all code from the files into an array
code_array = load_code_from_files(code_directory_path)
labels_array = load_labels_from_files(labels_directory_path)

# Step 5: Output the array size and the first element to check
print(f'Total code files loaded: {len(code_array)}')
print(code_array)

print(f'Total label files loaded: {len(labels_array)}')
print(labels_array)

../CPP_Files ['addrdb.cpp', 'addresstype.cpp', 'addrman.cpp', 'auto_updater.cc', 'command_line_args.cc', 'electron_content_client.cc', 'function_data.cpp', 'gpt_functions.cpp', 'gpt_se.cpp', 'gpt_sef.cpp', 'node_main.cc', 'uv_stdio_fix.cc']
../Labels ['addrdb.txt', 'addresstype.txt', 'addrman.txt', 'auto_updater.txt', 'command_line_args.txt', 'electron_content_client.txt', 'function_data.txt', 'gpt_functions.txt', 'gpt_se.txt', 'gpt_sef.txt', 'node_main.txt', 'uv_stdio_fix.txt']
Total code files loaded: 12
['// Copyright (c) 2009-2010 Satoshi Nakamoto\n// Copyright (c) 2009-2022 The Bitcoin Core developers\n// Distributed under the MIT software license, see the accompanying\n// file COPYING or http://www.opensource.org/licenses/mit-license.php.\n\n#include <bitcoin-build-config.h> // IWYU pragma: keep\n\n#include <addrdb.h>\n\n#include <addrman.h>\n#include <chainparams.h>\n#include <clientversion.h>\n#include <common/args.h>\n#include <common/settings.h>\n#include <cstdint>\n#include 

In [50]:
def print_node(node, root_node, code):
  print(f"Node: {node.type} - {code[node.start_byte:node.end_byte].decode('utf-8')}")

def find_global_variables(root_node, code):
    globals = set()
    const_variables = set()
    mutable_variables = set()

    # Recursively check for top-level declarations to detect global variables
    def _find_globals(node):
        if node.type == "declaration" and node.parent == root_node:
            var_name = code[node.start_byte:node.end_byte].decode('utf-8').split()[1]

            # Check if the variable is const or mutable
            declaration = code[node.start_byte:node.end_byte].decode('utf-8')
            if "const" in declaration:
                const_variables.add(var_name)
            elif "mutable" in declaration:
                mutable_variables.add(var_name)
            else:
                globals.add(var_name)

        for child in node.children:
            _find_globals(child)

    _find_globals(root_node)
    return globals, const_variables, mutable_variables

In [51]:
def find_local_variables(function_node, code):
    """
    Identifies local variables within a function.
    - Function parameters (by value).
    - Variables declared within the function body.
    """
    local_variables = set()
    referenced_parameters = set()

    # Extract function parameters
    declarator_node = function_node.child_by_field_name("declarator")
    if declarator_node and declarator_node.type == "function_declarator":
        parameters_node = declarator_node.child_by_field_name("parameters")
        if parameters_node:
            for param in parameters_node.children:
                if param.type == "parameter_declaration":
                    param_text = code[param.start_byte:param.end_byte].decode("utf-8").strip()
                    param_parts = param_text.split()
                    if len(param_parts) > 1:
                        param_name = param_parts[-1].strip("&")  # Exclude '&' for reference parameters
                        if "&" in param_text:
                            referenced_parameters.add(param_name)
                        else:
                            local_variables.add(param_name)

    # Traverse the function body for variable declarations
    body_node = function_node.child_by_field_name("body")
    if body_node:

        def add_var_name(var_name):
            if "{" in var_name:
                var_name = var_name.split('{')[0]
            if var_name == "":
                return
            if var_name[0] == '&':
                return
            if var_name[0] == '*':
                var_name = var_name[1:]
            if var_name[-1].isalnum() or var_name[-1] == '_':
                local_variables.add(var_name)
        def _find_declarations(node):
            if node.type == "declaration":
                declaration_text = code[node.start_byte:node.end_byte].decode("utf-8").strip()
                multi_declarations = declaration_text.split(',')
                for declaration in multi_declarations:
                    declaration = declaration.split('=')[0].strip().strip(';')
                    parts = declaration.split()
                    add_var_name(parts[-1])

            # Recursively check all children in the node
            for child in node.children:
                _find_declarations(child)

        _find_declarations(body_node)

    return local_variables, referenced_parameters

In [52]:
def has_side_effect(node, code, globals, const_variables, mutable_variables, function_local_variables, meta_reasoning_level):
    # Helper function to get the current function from the node
    def get_function_for_node(node):
        while node:
            if node.type == "function_definition":
                return node
            node = node.parent
        return None

    # Helper function to check if a variable is local to the current function
    def is_local(variable_name, current_function_node):
        if variable_name[0] == "*" or variable_name[0] == '&':
            variable_name = variable_name[1:]
        if "[" in variable_name:
            variable_name = variable_name.split("[")[0]
        if current_function_node:
            local_variables = function_local_variables.get(current_function_node, set())
            return variable_name in local_variables or variable_name in mutable_variables
        return False

    # Helper function to extract variable text from the left side of an expression
    def get_variable_text(left_side_node):
        return code[left_side_node.start_byte:left_side_node.end_byte].decode('utf-8') if left_side_node else None

    current_function_node = get_function_for_node(node)

    # Check if the return value of a function call is unused
    def is_unused_return(node):
        # The parent node determines if the return value is used
        if "back_inserter" in code[node.start_byte:node.end_byte].decode('utf-8'):
            print_node(node, root_node, code)
            print_node(node.parent, root_node, code)
        parent = node.parent
        if parent is None or parent.type == "expression_statement":
            return True  # No parent implies the return value is unused

        # The return value is used if the call is part of an assignment or another expression
        if parent.type in {
            "assignment_expression",
            "compound_assignment_expression",
            "return_statement",
            "binary_expression",
            "subscript_expression",
            "init_declarator",
            "preproc_if",
            "call_expression",
        }:
            return False
        # Otherwise, treat as unused
        return False

    # Check if a function call involves globals only
    def only_involves_globals(function_name, argument_nodes):
        if "." in function_name:
            for part in function_name.split("."):
                if is_local(part, current_function_node):
                    return False
        if "->" in function_name:
            for part in function_name.split("->"):
                if is_local(part, current_function_node):
                    return False
        if "unref" in function_name:
            print(argument_nodes)
        uses_only_globals = all(
            arg.type != "call_expression" and not is_local(get_variable_text(arg), current_function_node) for arg in argument_nodes if arg.type == "identifier" or arg.type == "pointer_expression" or arg.type == "call_expression"
        )
        if "unref" in function_name:
            print(uses_only_globals)
        return uses_only_globals

    # Handle assignments, including those involving pointer dereferencing (e.g., global_pointer->x = 5)
    if node.type in {"assignment_expression", "compound_assignment_expression"}:
        left_side = node.child_by_field_name("left")
        if left_side:
            # Handle member access (e.g., global_pointer->x, global_object.x)
            if left_side.type == "field_expression":
                object_node = left_side.children[0]
                field_node = left_side.children[2]
                object_text = get_variable_text(object_node)
                field_text = get_variable_text(field_node)
                if not is_local(object_text, current_function_node):
                    return True, f"Modification of field '{field_text}' through global '{object_text}'"
                else:
                    return False, None

            # Handle array modifications (e.g., array[index] = value)
            if left_side.type == "subscript_expression":
                array_name_node = left_side.children[0]
                if array_name_node:
                    array_name = get_variable_text(array_name_node)

                    if not is_local(array_name, current_function_node):
                        return True, f"Modification of global array '{array_name}'"
                    else:
                        return False, None

            # Handle scalar variable assignments
            left_text = get_variable_text(left_side)
            if left_text:
                # Const variables cannot be modified (but it's not a side effect per se)
                if left_text in const_variables:
                    return False, "Modification of const variable is not allowed"

                # Global or unseen variables being modified
                if not is_local(left_text, current_function_node):
                    return True, f"Modification of global variable '{left_text}'"
                else:
                    return False, None

    # Handle increment/decrement (e.g., global_var++, global_var-- or ++id, --id)
    if node.type == "update_expression":
        operand_node = None

        # Find the operand and the operator in the children of the update_expression node
        for child in node.children:
            if child.type == "identifier":
                operand_node = child  # The variable being updated

        # Extract the operand's text (variable name)
        operand_text = get_variable_text(operand_node)

        if operand_text:
            if not is_local(operand_text, current_function_node):
                return True, f"Update of global variable '{operand_text}'"
            else:
                return False, None

    # Handle throw statements
    if node.type == "throw_statement":
        exception_text = code[node.start_byte:node.end_byte].decode('utf-8').strip()
        return True, f"Throw statement detected: {exception_text}"

    # Handle exit calls as side effects
    if node.type == "call_expression":
        function_node = node.child_by_field_name("function")
        if function_node:
            function_name = get_variable_text(function_node)
            if function_name in {"exit", "_Exit", "abort"}:
                return True, f"Exit call detected: {function_name}"
            if meta_reasoning_level > 0:
                likely_SE_patterns = [
                    "exit", "Exit", "EXIT", "terminate", "Terminate", "TERMINATE",
                    "log", "Log",
                    "remove", "delete"
                ]
                if meta_reasoning_level > 1:
                    likely_SE_patterns += [">>", "<<", "print", "Print"]  # print is level 2 due to the existence of functions like strprintf which use print to signify formatting.
                    # >> and << are level 2 because they are often used for printing but sometimes for foramtting and sometimes for bit shifts.
                    argument_list_node = node.child_by_field_name("arguments")
                    argument_nodes = []
                    if argument_list_node:
                        argument_nodes = [child for child in argument_list_node.children if child.type != ","]
                    if is_unused_return(node) and only_involves_globals(function_name, argument_nodes):
                        return True, f"Function call that only uses globals and has unused return value: {function_name}"
                if meta_reasoning_level > 2:
                    likely_SE_patterns += ["set", "Set"]  # VERY strong assumption that functions with "set" in their names are side-effect producing
                if any(pattern in function_name for pattern in likely_SE_patterns):
                    return True, f"Likely side-effect producing function: {function_name} "

    # Check for I/O operations (e.g., std::cout, std::cin, file streams)
    if node.type in {"binary_expression", "call_expression"}:
        text = code[node.start_byte:node.end_byte].decode('utf-8')
        io_patterns = [
            "cout", "cerr", "cin",  # Standard I/O
            "fwrite",  # C-style file functions
            "ofstream",  # File streams
        ]
        if any(pattern in text for pattern in io_patterns):
            return True, "I/O operation"

    # Check for declarations that create I/O streams (e.g., std::ofstream) (not including ifstream since that's read only)
    if node.type == "declaration":
        text = code[node.start_byte:node.end_byte].decode('utf-8')
        if "ofstream" in text:
            return True, "File I/O stream creation"

    # No side effect detected
    return False, None


In [53]:
def side_effect_lines(node, root_node, code, meta_reasoning_level):
    """
    Detect lines with side effects, now considering local variables per function.
    """
    global_variables, const_variables, mutable_variables = find_global_variables(root_node, code)

    # Traverse all function definitions and track their local variables
    def _find_local_vars(node):
        if node.type == "function_definition":
            local_vars, referenced_params = find_local_variables(node, code)
            return local_vars, referenced_params
        for child in node.children:
            local_vars, referenced_params = _find_local_vars(child)
            if local_vars:
                return local_vars, referenced_params
        return set(), set()

    # Dictionary to store local variables per function
    function_local_variables = {}

    def _extract_local_vars(node):
        if node.type == "function_definition":
            local_vars, _ = find_local_variables(node, code)
            function_local_variables[node] = local_vars
        for child in node.children:
            _extract_local_vars(child)

    _extract_local_vars(root_node)
    print(function_local_variables)

    # Now we have a mapping of each function to its local variables
    lines = code.decode('utf-8').splitlines()
    side_effect_flags = [0] * len(lines)
    side_effect_reasons = [""] * len(lines)

    def _mark_side_effects(node):
        has_effect, reason = has_side_effect(node, code, global_variables, const_variables, mutable_variables, function_local_variables, meta_reasoning_level)
        if has_effect:
            start_line = node.start_point[0]
            end_line = node.end_point[0]
            reason_with_line = f"line {start_line + 1}: {reason}"
            for line in range(start_line, end_line + 1):
                side_effect_flags[line] = 1
                side_effect_reasons[line] = reason_with_line

        for child in node.children:
            _mark_side_effects(child)

    _mark_side_effects(node)
    return side_effect_flags, side_effect_reasons


In [54]:
import tree_sitter_cpp as tscpp
from tree_sitter import Language, Parser
parser = Parser(Language(tscpp.language()))

def labels_file_to_array(file):
    lines = file.splitlines()  # Read all lines
    # Check for BOM in the first line and remove it if present
    if lines[0].startswith("\ufeff"):
        lines[0] = lines[0][1:]  # Remove BOM from the first line
    return [int(line.strip()) for line in lines]

# Functions for metric calculations
def calc_precision(true_pos, false_pos):
    return true_pos / (true_pos + false_pos) if (true_pos + false_pos) > 0 else 0

def calc_recall(true_pos, false_neg):
    return true_pos / (true_pos + false_neg) if (true_pos + false_neg) > 0 else 0

def calc_f1_score(true_pos, false_pos, false_neg):
    precision = calc_precision(true_pos, false_pos)
    recall = calc_recall(true_pos, false_neg)
    return (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def calc_accuracy(true_pos, true_neg, total):
    return (true_pos + true_neg) / total if total > 0 else 0

In [55]:
# Initialize totals
total_true_pos = 0
total_true_neg = 0
total_false_pos = 0
total_false_neg = 0
total = 0

META_REASONING_LEVEL = 3
for code, labels, file_name in zip(code_array, labels_array, file_names):
    print("============================================")
    print(f"FILE: {file_name}")
    # Parse the code
    code = code.encode("utf-8")
    tree = parser.parse(code)

    # Access the root node of the parsed syntax tree
    root_node = tree.root_node
    side_effect_flags, side_effect_reasons = side_effect_lines(root_node, root_node, code, META_REASONING_LEVEL)
    labels = labels_file_to_array(labels)
    if len(side_effect_flags) + 1 == len(labels) and labels[-1] == 0:  # Mistake in labeling the EOF line sometimes.
        labels = labels[:-1]
    assert(len(side_effect_flags) == len(labels))
    lines = code.decode('utf-8').splitlines()

    # Per-file counters
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0

    for i, (predict, actual, reason, line) in enumerate(zip(side_effect_flags, labels, side_effect_reasons, lines)):
        if predict == 0:
            if actual == 0:
                true_neg += 1
            else:
                print(f"False Negative, Line: {i+1},      Code: {line}")
                false_neg += 1
        else:
            if actual == 0:
                print(f"False Positive, Line: {i+1},      Code: {line},       //Reason: {reason}")
                false_pos += 1
            else:
                print(f"True Positive, Line: {i+1},       Code: {line},       //Reason: {reason}")
                true_pos += 1

    # File-level results
    print("-------------------------------------------")
    print(f"FOR FILE: {file_name}")
    print(f"True positives: {true_pos} ({(true_pos / len(lines) * 100):.0f}%)")
    print(f"True negatives: {true_neg} ({(true_neg / len(lines) * 100):.0f}%)")
    print(f"False positives: {false_pos} ({(false_pos / len(lines) * 100):.0f}%)")
    print(f"False negatives: {false_neg} ({(false_neg / len(lines) * 100):.0f}%)")
    print(f"Accuracy: {calc_accuracy(true_pos, true_neg, len(lines))}")
    print(f"Precision: {calc_precision(true_pos, false_pos)}")
    print(f"Recall: {calc_recall(true_pos, false_neg)}")
    print(f"F1 Score: {calc_f1_score(true_pos, false_pos, false_neg)}")

    # Update totals
    total_true_pos += true_pos
    total_true_neg += true_neg
    total_false_pos += false_pos
    total_false_neg += false_neg
    total += len(lines)

# Overall results
print("============================================")
print("OVERALL RESULTS:")
print(f"True positives: {total_true_pos} ({(total_true_pos / total * 100):.0f}%)")
print(f"True negatives: {total_true_neg} ({(total_true_neg / total * 100):.0f}%)")
print(f"False positives: {total_false_pos} ({(total_false_pos / total * 100):.0f}%)")
print(f"False negatives: {total_false_neg} ({(total_false_neg / total * 100):.0f}%)")
print(f"Accuracy: {calc_accuracy(total_true_pos, total_true_neg, total)}")
print(f"Precision: {calc_precision(total_true_pos, total_false_pos)}")
print(f"Recall: {calc_recall(total_true_pos, total_false_neg)}")
print(f"F1 Score: {calc_f1_score(total_true_pos, total_false_pos, total_false_neg)}")



FILE: addrdb
{<Node type=function_definition, start_point=(36, 0), end_point=(49, 1)>: {'hashwriter'}, <Node type=function_definition, start_point=(52, 0), end_point=(91, 1)>: {'fileout', 'pathTmp', 'file', 'randv', 'prefix', 'tmpfn'}, <Node type=function_definition, start_point=(94, 0), end_point=(116, 1)>: {'pchMsgTmp', 'verifier', 'hashTmp'}, <Node type=function_definition, start_point=(119, 0), end_point=(127, 1)>: {'file', 'filein'}, <Node type=function_definition, start_point=(130, 0), end_point=(134, 1)>: {'ban_list_path'}, <Node type=function_definition, start_point=(136, 0), end_point=(147, 1)>: {'errors'}, <Node type=function_definition, start_point=(149, 0), end_point=(177, 1)>: {'errors', 'std::map<std::string', 'settings'}, <Node type=function_definition, start_point=(179, 0), end_point=(183, 1)>: {'pathAddr'}, <Node type=function_definition, start_point=(185, 0), end_point=(188, 1)>: set(), <Node type=function_definition, start_point=(190, 0), end_point=(220, 1)>: {'0', '

In [56]:
def load_from_file(file, code_array, labels_array, names_array):
    base_directory_path = '../dev_test_set'
    directory_path = os.path.join(base_directory_path, file, 'CPP_Files')

    for root, _, files in os.walk(directory_path):
        file_path = os.path.join(root, files[0])
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            code_array.append(content)

    labels_path = os.path.join(base_directory_path, file, 'Labels', file+'.txt')
    with open(labels_path, 'r', encoding='utf-8') as f:
        content = f.read()
        labels_array.append(content)
    names_array.append(file)

test_code = []
test_labels = []
test_files = []
load_from_file('ASTNode', test_code, test_labels, test_files)
load_from_file('c_api_function', test_code, test_labels, test_files)
load_from_file('ModuleDiagsConsumer', test_code, test_labels, test_files)
load_from_file('NativeCxxModuleExample', test_code, test_labels, test_files)
load_from_file('register_core_types', test_code, test_labels, test_files)

In [57]:
print(test_code)
print(test_files)
print(test_labels)

['//===--- ASTNode.cpp - Swift Language ASTs --------------------------------===//\n//\n// This source file is part of the Swift.org open source project\n//\n// Copyright (c) 2014 - 2017 Apple Inc. and the Swift project authors\n// Licensed under Apache License v2.0 with Runtime Library Exception\n//\n// See https://swift.org/LICENSE.txt for license information\n// See https://swift.org/CONTRIBUTORS.txt for the list of Swift project authors\n//\n//===----------------------------------------------------------------------===//\n//\n// This file implements the ASTNode, which is a union of Stmt, Expr, and Decl.\n//\n//===----------------------------------------------------------------------===//\n\n#include "swift/AST/ASTNode.h"\n#include "swift/AST/Decl.h"\n#include "swift/AST/Expr.h"\n#include "swift/AST/Stmt.h"\n#include "swift/AST/Pattern.h"\n#include "swift/AST/TypeRepr.h"\n#include "swift/Basic/Assertions.h"\n#include "swift/Basic/SourceLoc.h"\n#include "swift/Parse/Token.h"\n\nusing

In [58]:
def do_test(meta_reasoning_level):
    # Initialize totals
    test_total_true_pos = 0
    test_total_true_neg = 0
    test_total_false_pos = 0
    test_total_false_neg = 0
    test_total = 0
    test_flags = []

    for code, labels, file_name in zip(test_code, test_labels, test_files):
        print("============================================")
        print(f"FILE: {file_name}")
        # Parse the code
        code = code.encode("utf-8")
        tree = parser.parse(code)

        # Access the root node of the parsed syntax tree
        root_node = tree.root_node
        side_effect_flags, side_effect_reasons = side_effect_lines(root_node, root_node, code, meta_reasoning_level)
        test_flags.append(side_effect_flags)
        labels = labels_file_to_array(labels)
        if len(side_effect_flags) + 1 == len(labels) and labels[-1] == 0:  # Mistake in labeling the EOF line sometimes.
            labels = labels[:-1]
        assert(len(side_effect_flags) == len(labels))
        lines = code.decode('utf-8').splitlines()

        # Per-file counters
        true_pos = 0
        true_neg = 0
        false_pos = 0
        false_neg = 0

        for i, (predict, actual, reason, line) in enumerate(zip(side_effect_flags, labels, side_effect_reasons, lines)):
            if predict == 0:
                if actual == 0:
                    true_neg += 1
                else:
                    print(f"False Negative, Line: {i+1},      Code: {line}")
                    false_neg += 1
            else:
                if actual == 0:
                    print(f"False Positive, Line: {i+1},      Code: {line},       //Reason: {reason}")
                    false_pos += 1
                else:
                    print(f"True Positive, Line: {i+1},       Code: {line},       //Reason: {reason}")
                    true_pos += 1

        # File-level results
        print("-------------------------------------------")
        print(f"FOR FILE: {file_name}")
        print(f"True positives: {true_pos} ({(true_pos / len(lines) * 100):.0f}%)")
        print(f"True negatives: {true_neg} ({(true_neg / len(lines) * 100):.0f}%)")
        print(f"False positives: {false_pos} ({(false_pos / len(lines) * 100):.0f}%)")
        print(f"False negatives: {false_neg} ({(false_neg / len(lines) * 100):.0f}%)")
        print(f"Accuracy: {calc_accuracy(true_pos, true_neg, len(lines))}")
        print(f"Precision: {calc_precision(true_pos, false_pos)}")
        print(f"Recall: {calc_recall(true_pos, false_neg)}")
        print(f"F1 Score: {calc_f1_score(true_pos, false_pos, false_neg)}")

        # Update totals
        test_total_true_pos += true_pos
        test_total_true_neg += true_neg
        test_total_false_pos += false_pos
        test_total_false_neg += false_neg
        test_total += len(lines)

    # Overall results
    print("============================================")
    print("OVERALL RESULTS:")
    print(f"True positives: {test_total_true_pos} ({(test_total_true_pos / test_total * 100):.0f}%)")
    print(f"True negatives: {test_total_true_neg} ({(test_total_true_neg / test_total * 100):.0f}%)")
    print(f"False positives: {test_total_false_pos} ({(test_total_false_pos / test_total * 100):.0f}%)")
    print(f"False negatives: {test_total_false_neg} ({(test_total_false_neg / test_total * 100):.0f}%)")
    print(f"Accuracy: {calc_accuracy(test_total_true_pos, test_total_true_neg, test_total)}")
    print(f"Precision: {calc_precision(test_total_true_pos, test_total_false_pos)}")
    print(f"Recall: {calc_recall(test_total_true_pos, test_total_false_neg)}")
    print(f"F1 Score: {calc_f1_score(test_total_true_pos, test_total_false_pos, test_total_false_neg)}")

    return test_flags

In [59]:
static_predictions_0 = do_test(0)

FILE: ASTNode
{<Node type=function_definition, start_point=(28, 0), end_point=(46, 1)>: {'C', 'T', 'D', 'E', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(49, 0), end_point=(51, 1)>: set(), <Node type=function_definition, start_point=(54, 0), end_point=(56, 1)>: set(), <Node type=function_definition, start_point=(58, 0), end_point=(70, 1)>: {'D', 'E'}, <Node type=function_definition, start_point=(72, 0), end_point=(88, 1)>: {'C', 'T', 'D', 'E', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(90, 0), end_point=(111, 1)>: {'C', 'T', 'D', 'E', 'G', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(113, 0), end_point=(132, 1)>: {'C', 'T', 'D', 'E', 'Indent', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(134, 0), end_point=(136, 1)>: set(), <Node type=function_definition, start_point=(138, 0), end_point=(146, 1)>: {'kind'}}
False Negative, Line: 46,      Code:   llvm_unreachable("unsupported AST node");
False Negative, Line: 69,      Cod

In [60]:
static_predictions_1 = do_test(1)

FILE: ASTNode
{<Node type=function_definition, start_point=(28, 0), end_point=(46, 1)>: {'C', 'T', 'D', 'E', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(49, 0), end_point=(51, 1)>: set(), <Node type=function_definition, start_point=(54, 0), end_point=(56, 1)>: set(), <Node type=function_definition, start_point=(58, 0), end_point=(70, 1)>: {'D', 'E'}, <Node type=function_definition, start_point=(72, 0), end_point=(88, 1)>: {'C', 'T', 'D', 'E', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(90, 0), end_point=(111, 1)>: {'C', 'T', 'D', 'E', 'G', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(113, 0), end_point=(132, 1)>: {'C', 'T', 'D', 'E', 'Indent', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(134, 0), end_point=(136, 1)>: set(), <Node type=function_definition, start_point=(138, 0), end_point=(146, 1)>: {'kind'}}
False Negative, Line: 46,      Code:   llvm_unreachable("unsupported AST node");
False Negative, Line: 69,      Cod

In [61]:
static_predictions_2 = do_test(2)

FILE: ASTNode
{<Node type=function_definition, start_point=(28, 0), end_point=(46, 1)>: {'C', 'T', 'D', 'E', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(49, 0), end_point=(51, 1)>: set(), <Node type=function_definition, start_point=(54, 0), end_point=(56, 1)>: set(), <Node type=function_definition, start_point=(58, 0), end_point=(70, 1)>: {'D', 'E'}, <Node type=function_definition, start_point=(72, 0), end_point=(88, 1)>: {'C', 'T', 'D', 'E', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(90, 0), end_point=(111, 1)>: {'C', 'T', 'D', 'E', 'G', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(113, 0), end_point=(132, 1)>: {'C', 'T', 'D', 'E', 'Indent', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(134, 0), end_point=(136, 1)>: set(), <Node type=function_definition, start_point=(138, 0), end_point=(146, 1)>: {'kind'}}
False Positive, Line: 45,      Code:   assert(!isNull() && "Null ASTNode doesn't have a source range");,       //Re

In [62]:
static_predictions_3 = do_test(3)

FILE: ASTNode
{<Node type=function_definition, start_point=(28, 0), end_point=(46, 1)>: {'C', 'T', 'D', 'E', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(49, 0), end_point=(51, 1)>: set(), <Node type=function_definition, start_point=(54, 0), end_point=(56, 1)>: set(), <Node type=function_definition, start_point=(58, 0), end_point=(70, 1)>: {'D', 'E'}, <Node type=function_definition, start_point=(72, 0), end_point=(88, 1)>: {'C', 'T', 'D', 'E', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(90, 0), end_point=(111, 1)>: {'C', 'T', 'D', 'E', 'G', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(113, 0), end_point=(132, 1)>: {'C', 'T', 'D', 'E', 'Indent', 'I', 'P', 'S'}, <Node type=function_definition, start_point=(134, 0), end_point=(136, 1)>: set(), <Node type=function_definition, start_point=(138, 0), end_point=(146, 1)>: {'kind'}}
False Positive, Line: 45,      Code:   assert(!isNull() && "Null ASTNode doesn't have a source range");,       //Re

In [63]:
#print(f"Accuracy: {calc_accuracy(test_total_true_pos+total_true_pos, test_total_true_neg+total_true_neg, test_total+total)}")
#print(f"Precision: {calc_precision(test_total_true_pos+total_true_pos, test_total_false_pos+total_false_pos)}")
#print(f"Recall: {calc_recall(test_total_true_pos+total_true_pos, test_total_false_neg+total_false_neg)}")
#print(f"F1 Score: {calc_f1_score(test_total_true_pos+total_true_pos, test_total_false_pos+total_false_neg, test_total_false_neg+total_false_neg)}")

In [64]:
bert_model_ASTNode = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
bert_model_c_api_function_line = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
bert_model_ModuleDiagsConsumer = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
bert_model_NativeCxxModuleExample = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
bert_model_register_core_types = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0]

bert_model = [bert_model_ASTNode, bert_model_c_api_function_line, bert_model_ModuleDiagsConsumer, bert_model_NativeCxxModuleExample, bert_model_register_core_types]


In [65]:
def metrics(predicted_array, actual_array, name_array):
    total_true_pos = 0
    total_true_neg = 0
    total_false_pos = 0
    total_false_neg = 0
    total = 0

    for predicted, actual, file_name in zip(predicted_array, actual_array, name_array):
        actual = labels_file_to_array(actual)
        print("============================================")
        print(f"FILE: {file_name}")

        true_pos = 0
        true_neg = 0
        false_pos = 0
        false_neg = 0

        if len(predicted) + 1 == len(actual) and actual[-1] == 0:  # Mistake in labeling the EOF line sometimes.
            actual = actual[:-1]
        
        l = len(actual)
        assert(l == len(predicted))

        for p, a in zip(predicted, actual):
            if p == 0:
                if a == 0:
                    true_neg += 1
                else:
                    false_neg += 1
            else:
                if a == 0:
                    false_pos += 1
                else:
                    true_pos += 1

        print(f"True positives: {true_pos} ({(true_pos / l * 100):.0f}%)")
        print(f"True negatives: {true_neg} ({(true_neg / l * 100):.0f}%)")
        print(f"False positives: {false_pos} ({(false_pos / l * 100):.0f}%)")
        print(f"False negatives: {false_neg} ({(false_neg / l * 100):.0f}%)")
        print(f"Accuracy: {calc_accuracy(true_pos, true_neg, l)}")
        print(f"Precision: {calc_precision(true_pos, false_pos)}")
        print(f"Recall: {calc_recall(true_pos, false_neg)}")
        print(f"F1 Score: {calc_f1_score(true_pos, false_pos, false_neg)}")

        # Update totals
        total_true_pos += true_pos
        total_true_neg += true_neg
        total_false_pos += false_pos
        total_false_neg += false_neg
        total += l

    # Overall results
    print("============================================")
    print("OVERALL RESULTS:")
    print(f"True positives: {total_true_pos} ({(total_true_pos / total * 100):.0f}%)")
    print(f"True negatives: {total_true_neg} ({(total_true_neg / total * 100):.0f}%)")
    print(f"False positives: {total_false_pos} ({(total_false_pos / total * 100):.0f}%)")
    print(f"False negatives: {total_false_neg} ({(total_false_neg / total * 100):.0f}%)")
    print(f"Accuracy: {calc_accuracy(total_true_pos, total_true_neg, total)}")
    print(f"Precision: {calc_precision(total_true_pos, total_false_pos)}")
    print(f"Recall: {calc_recall(total_true_pos, total_false_neg)}")
    print(f"F1 Score: {calc_f1_score(total_true_pos, total_false_pos, total_false_neg)}")

In [66]:
metrics(bert_model, test_labels, test_files)

FILE: ASTNode
True positives: 14 (9%)
True negatives: 137 (86%)
False positives: 0 (0%)
False negatives: 8 (5%)
Accuracy: 0.949685534591195
Precision: 1.0
Recall: 0.6363636363636364
F1 Score: 0.7777777777777778
FILE: c_api_function
True positives: 31 (8%)
True negatives: 286 (78%)
False positives: 16 (4%)
False negatives: 32 (9%)
Accuracy: 0.8684931506849315
Precision: 0.6595744680851063
Recall: 0.49206349206349204
F1 Score: 0.5636363636363635
FILE: ModuleDiagsConsumer
True positives: 12 (7%)
True negatives: 154 (88%)
False positives: 5 (3%)
False negatives: 5 (3%)
Accuracy: 0.9431818181818182
Precision: 0.7058823529411765
Recall: 0.7058823529411765
F1 Score: 0.7058823529411765
FILE: NativeCxxModuleExample
True positives: 14 (6%)
True negatives: 213 (87%)
False positives: 3 (1%)
False negatives: 14 (6%)
Accuracy: 0.930327868852459
Precision: 0.8235294117647058
Recall: 0.5
F1 Score: 0.6222222222222222
FILE: register_core_types
True positives: 213 (47%)
True negatives: 225 (49%)
False po

In [67]:
def combine_results(a_results, b_results):
    results = []
    for a_result, b_result in zip(a_results, b_results):
        arr = []
        for a, b in zip(a_result, b_result):
            if a == b:
                arr.append(a)
            else:
                arr.append(1)
        results.append(arr)
    return results

In [68]:
metrics(combine_results(bert_model, static_predictions_0), test_labels, test_files)

FILE: ASTNode
True positives: 14 (9%)
True negatives: 137 (86%)
False positives: 0 (0%)
False negatives: 8 (5%)
Accuracy: 0.949685534591195
Precision: 1.0
Recall: 0.6363636363636364
F1 Score: 0.7777777777777778
FILE: c_api_function
True positives: 39 (11%)
True negatives: 283 (78%)
False positives: 19 (5%)
False negatives: 24 (7%)
Accuracy: 0.8821917808219178
Precision: 0.6724137931034483
Recall: 0.6190476190476191
F1 Score: 0.6446280991735537
FILE: ModuleDiagsConsumer
True positives: 13 (7%)
True negatives: 154 (88%)
False positives: 5 (3%)
False negatives: 4 (2%)
Accuracy: 0.9488636363636364
Precision: 0.7222222222222222
Recall: 0.7647058823529411
F1 Score: 0.7428571428571428
FILE: NativeCxxModuleExample
True positives: 14 (6%)
True negatives: 213 (87%)
False positives: 3 (1%)
False negatives: 14 (6%)
Accuracy: 0.930327868852459
Precision: 0.8235294117647058
Recall: 0.5
F1 Score: 0.6222222222222222
FILE: register_core_types
True positives: 216 (47%)
True negatives: 225 (49%)
False po

In [69]:
metrics(combine_results(bert_model, static_predictions_1), test_labels, test_files)

FILE: ASTNode
True positives: 14 (9%)
True negatives: 137 (86%)
False positives: 0 (0%)
False negatives: 8 (5%)
Accuracy: 0.949685534591195
Precision: 1.0
Recall: 0.6363636363636364
F1 Score: 0.7777777777777778
FILE: c_api_function
True positives: 39 (11%)
True negatives: 283 (78%)
False positives: 19 (5%)
False negatives: 24 (7%)
Accuracy: 0.8821917808219178
Precision: 0.6724137931034483
Recall: 0.6190476190476191
F1 Score: 0.6446280991735537
FILE: ModuleDiagsConsumer
True positives: 13 (7%)
True negatives: 154 (88%)
False positives: 5 (3%)
False negatives: 4 (2%)
Accuracy: 0.9488636363636364
Precision: 0.7222222222222222
Recall: 0.7647058823529411
F1 Score: 0.7428571428571428
FILE: NativeCxxModuleExample
True positives: 14 (6%)
True negatives: 213 (87%)
False positives: 3 (1%)
False negatives: 14 (6%)
Accuracy: 0.930327868852459
Precision: 0.8235294117647058
Recall: 0.5
F1 Score: 0.6222222222222222
FILE: register_core_types
True positives: 216 (47%)
True negatives: 225 (49%)
False po

In [70]:
metrics(combine_results(bert_model, static_predictions_2), test_labels, test_files)

FILE: ASTNode
True positives: 14 (9%)
True negatives: 132 (83%)
False positives: 5 (3%)
False negatives: 8 (5%)
Accuracy: 0.9182389937106918
Precision: 0.7368421052631579
Recall: 0.6363636363636364
F1 Score: 0.6829268292682926
FILE: c_api_function
True positives: 45 (12%)
True negatives: 283 (78%)
False positives: 19 (5%)
False negatives: 18 (5%)
Accuracy: 0.8986301369863013
Precision: 0.703125
Recall: 0.7142857142857143
F1 Score: 0.7086614173228346
FILE: ModuleDiagsConsumer
True positives: 15 (9%)
True negatives: 154 (88%)
False positives: 5 (3%)
False negatives: 2 (1%)
Accuracy: 0.9602272727272727
Precision: 0.75
Recall: 0.8823529411764706
F1 Score: 0.8108108108108107
FILE: NativeCxxModuleExample
True positives: 26 (11%)
True negatives: 210 (86%)
False positives: 6 (2%)
False negatives: 2 (1%)
Accuracy: 0.9672131147540983
Precision: 0.8125
Recall: 0.9285714285714286
F1 Score: 0.8666666666666666
FILE: register_core_types
True positives: 228 (50%)
True negatives: 225 (49%)
False positi

In [71]:
metrics(combine_results(bert_model, static_predictions_3), test_labels, test_files)

FILE: ASTNode
True positives: 14 (9%)
True negatives: 132 (83%)
False positives: 5 (3%)
False negatives: 8 (5%)
Accuracy: 0.9182389937106918
Precision: 0.7368421052631579
Recall: 0.6363636363636364
F1 Score: 0.6829268292682926
FILE: c_api_function
True positives: 45 (12%)
True negatives: 283 (78%)
False positives: 19 (5%)
False negatives: 18 (5%)
Accuracy: 0.8986301369863013
Precision: 0.703125
Recall: 0.7142857142857143
F1 Score: 0.7086614173228346
FILE: ModuleDiagsConsumer
True positives: 15 (9%)
True negatives: 154 (88%)
False positives: 5 (3%)
False negatives: 2 (1%)
Accuracy: 0.9602272727272727
Precision: 0.75
Recall: 0.8823529411764706
F1 Score: 0.8108108108108107
FILE: NativeCxxModuleExample
True positives: 26 (11%)
True negatives: 210 (86%)
False positives: 6 (2%)
False negatives: 2 (1%)
Accuracy: 0.9672131147540983
Precision: 0.8125
Recall: 0.9285714285714286
F1 Score: 0.8666666666666666
FILE: register_core_types
True positives: 228 (50%)
True negatives: 225 (49%)
False positi

In [72]:
combine_results(bert_model, static_predictions_3)[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0]

In [73]:
print(total)

2876


In [74]:
bad_bert = []
small_bert = []
bad_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
small_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
bad_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
small_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
bad_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
small_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
bad_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
small_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
bad_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
small_bert.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])



In [ ]:
newfile = """#include "dlhandle.h"

#include <string>

#ifndef _WIN32
#   include <dlfcn.h>
#else
#   include <cassert>
#   include <sstream>
#   define WIN32_LEAN_AND_MEAN
#   ifndef NOMINMAX
#       define NOMINMAX
#   endif
#   include <windows.h>
#endif

using namespace std::string_literals
namespace fs = std::filesystem


#ifndef _WIN32

Dlhandle::Dlhandle(const fs::path &fpath)
{
    chandle = dlopen(fpath.c_str(), RTLD_LAZY | RTLD_LOCAL)
    if (!chandle) {
        throw Exception("dlopen: "s + dlerror())
    }
}

Dlhandle::~Dlhandle()
{
    if (chandle) dlclose(chandle)
}

void *Dlhandle::get_internal(const char *symbol) const
{
    return dlsym(chandle, symbol)
}

#else // defined(_WIN32)

Dlhandle::Dlhandle(const fs::path &fpath)
{
    fs::path afpath = fs::absolute(fpath)

    // Suppress the "Entry Point Not Found" dialog, caused by outdated nvcuda.dll from the GPU driver
    UINT lastErrorMode = GetErrorMode()
    SetErrorMode(lastErrorMode | SEM_FAILCRITICALERRORS)

    chandle = LoadLibraryExW(afpath.c_str(), NULL, LOAD_LIBRARY_SEARCH_DEFAULT_DIRS | LOAD_LIBRARY_SEARCH_DLL_LOAD_DIR)

    SetErrorMode(lastErrorMode)

    if (!chandle) {
        DWORD err = GetLastError()
        std::ostringstream ss
        ss << "LoadLibraryExW failed with error 0x" << std::hex << err
        throw Exception(ss.str())
    }
}

Dlhandle::~Dlhandle()
{
    if (chandle) FreeLibrary(HMODULE(chandle))
}

void *Dlhandle::get_internal(const char *symbol) const
{
    return GetProcAddress(HMODULE(chandle), symbol)
}

#endif // defined(_WIN32)
"""

In [92]:
code = newfile.encode("utf-8")
tree = parser.parse(code)

# Access the root node of the parsed syntax tree
root_node = tree.root_node
flags, reasons = side_effect_lines(root_node, root_node, code, 3)

{<Node type=function_definition, start_point=(22, 0), end_point=(28, 1)>: set(), <Node type=function_definition, start_point=(30, 0), end_point=(33, 1)>: set(), <Node type=function_definition, start_point=(35, 0), end_point=(38, 1)>: set(), <Node type=function_definition, start_point=(62, 0), end_point=(65, 1)>: set(), <Node type=function_definition, start_point=(67, 0), end_point=(70, 1)>: set()}


In [93]:
for flag, reason in zip(flags, reasons):
    if flag == 1:
        print(reason)

line 25: Modification of global variable 'chandle'
line 27: Throw statement detected: throw Exception("dlopen: "s + dlerror())
line 33: Function call that only uses globals and has unused return value: dlclose
line 49: Likely side-effect producing function: SetErrorMode 
line 49: Modification of global variable 'SetErrorMode(lastErrorMode | SEM_FAILCRITICALERRORS)'
line 49: Modification of global variable 'SetErrorMode(lastErrorMode | SEM_FAILCRITICALERRORS)'
line 53: Function call that only uses globals and has unused return value: SetErrorMode
